<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.2 MB/s eta 0:00:00


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
from xgboost import XGBClassifier


from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression


# warningsを非表示にする
warnings.filterwarnings("ignore")

In [16]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/csv_data/dft_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/csv_data/dft_test.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [17]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [18]:
test_x = df_test
test_x = test_x.drop(target, axis=1)

# モデル


In [19]:
params_lgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [20]:
params_cat = {
    "iterations": 3000,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [68]:
params_xgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "early_stopping_rounds": 100,
    "random_state": 0,
}

In [61]:
base_models = {
    'LightGBM': lgb.LGBMClassifier(**params_lgb),
    'CatBoost': CatBoostClassifier(**params_cat, early_stopping_rounds=100),
    'XGBoost': XGBClassifier(**params_xgb)
}

In [69]:
def train_base_models(X, y, models, cols_category, n_splits=5):
    meta_features = np.zeros((len(X), len(models)))
    oof_predictions = np.zeros(len(X))

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=815)

    for i, (model_name, model) in enumerate(models.items()):
        print(f"Training {model_name}...")
        oof_predictions_model = np.zeros(len(X))

        for fold, (trn_idx, val_idx) in enumerate(cv.split(X, y), start=1):
            trn_x, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
            val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]

            if isinstance(model, lgb.LGBMClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    callbacks=[lgb.early_stopping(100, verbose=False)],
                    categorical_feature=cols_category,
                )
            elif isinstance(model, CatBoostClassifier):
                train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
                val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
                model.fit(train_pool, eval_set=val_pool, verbose=False)
            elif isinstance(model, XGBClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    verbose=False,
                )
            else:
                model.fit(trn_x, trn_y)

            oof_predictions_model[val_idx] = model.predict_proba(val_x)[:, 1]

            auc = roc_auc_score(val_y, oof_predictions_model[val_idx])
            print(f"  Fold {fold}, AUC: {auc:.4f}")

        meta_features[:, i] = oof_predictions_model
        oof_predictions += oof_predictions_model / len(models)

        auc = roc_auc_score(y, oof_predictions_model)
        print(f"{model_name} OOF AUC: {auc:.4f}")

    return meta_features, oof_predictions

In [63]:
def train_meta_model(meta_features, y, meta_model, n_splits=5):
    oof_predictions = np.zeros(len(y))
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=815)

    for fold, (trn_idx, val_idx) in enumerate(cv.split(meta_features, y), start=1):
        trn_x, trn_y = meta_features[trn_idx], y.iloc[trn_idx]
        val_x, val_y = meta_features[val_idx], y.iloc[val_idx]

        meta_model.fit(trn_x, trn_y)
        oof_predictions[val_idx] = meta_model.predict_proba(val_x)[:, 1]

        auc = roc_auc_score(val_y, oof_predictions[val_idx])
        print(f"Meta Model Fold {fold}, AUC: {auc:.4f}")

    auc = roc_auc_score(y, oof_predictions)
    print(f"Meta Model OOF AUC: {auc:.4f}")

    return oof_predictions

In [64]:
def stacking_predict(X, base_models, meta_model):
    meta_features = np.column_stack([model.predict_proba(X)[:, 1] for _, model in base_models.items()])
    return meta_model.predict_proba(meta_features)[:, 1]

In [70]:
# ベースモデルの学習とメタ特徴量の生成
meta_features, oof_predictions_base = train_base_models(train_x, train_y, base_models, cols_category)


Training LightGBM...
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 776
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
  Fold 1, AUC: 0.8227
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 776
[LightGBM] [Info] Number of data points in the train set: 2791, number of use

In [71]:
meta_features

array([[8.38318243e-01, 6.75813024e-01, 9.32537973e-01],
       [1.37491267e-01, 1.30641515e-01, 1.99305248e-02],
       [3.21204616e-01, 2.54288914e-01, 2.60010272e-01],
       ...,
       [3.54496373e-02, 2.99904026e-02, 8.83868721e-04],
       [1.69734821e-01, 2.23324334e-01, 9.98414755e-02],
       [4.87300328e-02, 9.68759429e-02, 3.68798617e-03]])

In [72]:
params_logistic = {
    "penalty": 'l2',             # L2正則化
    "C": 0.5,                    # 正則化の強さ
    "solver": 'lbfgs',           # 最適化アルゴリズム
    "max_iter": 200,             # 最大反復回数
    "class_weight":'balanced',    # クラスの重みづけ
    "random_state": 0            # 乱数シード
}

In [74]:
# メタモデルの定義と学習
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression(**params_logistic)
oof_predictions_meta = train_meta_model(meta_features, train_y, meta_model)


Meta Model Fold 1, AUC: 0.8220
Meta Model Fold 2, AUC: 0.8449
Meta Model Fold 3, AUC: 0.8438
Meta Model Fold 4, AUC: 0.8237
Meta Model Fold 5, AUC: 0.8515
Meta Model OOF AUC: 0.8357


In [75]:
final_auc = roc_auc_score(train_y, oof_predictions_meta)
print(f"Final Stacking Model OOF AUC: {final_auc:.8f}")

Final Stacking Model OOF AUC: 0.83574240


In [76]:
base_models

{'LightGBM': LGBMClassifier(colsample_bytree=0.8, learning_rate=0.01, n_estimators=3000,
                random_seed=0, subsample=0.8, subsample_freq=1),
 'CatBoost': <catboost.core.CatBoostClassifier at 0x7f4843f9c040>,
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.8, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.01, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=3000, n_jobs=None,
               num_parallel_tree=None, random_state=0, ...)}

In [77]:
base_models.values()

dict_values([LGBMClassifier(colsample_bytree=0.8, learning_rate=0.01, n_estimators=3000,
               random_seed=0, subsample=0.8, subsample_freq=1), <catboost.core.CatBoostClassifier object at 0x7f4843f9c040>, XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3000, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)])

In [78]:
test_predictions = stacking_predict(test_x, base_models, meta_model)

In [34]:
test_predictions

array([0.0801132 , 0.10713224, 0.16697836, ..., 0.69521919, 0.15399334,
       0.0541278 ])

In [79]:
test_predictions

array([0.33008663, 0.41308765, 0.55117781, ..., 0.94304322, 0.52771905,
       0.23618157])

In [82]:
count = 2

In [83]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightcatxgb_en_{timestamp}_{count:03d}.csv"
ss[1] = test_predictions
ss.to_csv(file_name, header=False, index=False)